In [1]:
import numpy as np
from os import path, listdir, makedirs
from itertools import product
from brainpipe.system import study

### Compute Power across time
    No baseline correction 

In [46]:
from brainpipe.feat.utils._feat import _manageWindow
from brainpipe.feature import power
from mne.filter import resample

st = study('Olfacto')

################################################################################################
# Define power settings :
bsl, width, step = None, 6, 6 #358, 51 #[(768,1024),(1024,1280)],[(1280,1536)]
baseline = 'No baseline' #['-500_0'] #['-1500_-1000','-1000_-500'] #'700ms'
################################################################################################
reps = ['Encoding_By_Odor']

for rep in reps:
    path2save = path.join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
    code = 'E' if rep.startswith('Encoding') else 'R'
    ###############################################################################
    if not path.exists(path2save):
        makedirs(path2save)
    ###############################################################################  
    files = [k for k in st.search('bipo_all_noWM_physFT.npz', folder=('database/'+rep+'/'))]
    for fi in files:
        # Dict with all power features
        kwargs = {} # Define an empty dictionnary to save all power parameters
        kwargs['f'] = [[2,4],[4,14],[13,30],[30,120]] # Frequency vector
        fname = ['low_theta', 'high_theta', 'beta', 'BBG'] # Name of each frequencykwargs['f'] = [[3,7], [8,13], [13,30], [30,120]] # Frequency vector[0.1,1.5],[2,3]
        kwargs['split'] = [None,None,None,9]
        kwargs['width'], kwargs['step'] = width, step # take power in 358 samples (700ms) windows width every 51 samples (100ms)

        # Load file :
        loadname = path.join(st.path, 'database/'+rep+'/', fi)
        mat = np.load(loadname,allow_pickle=True)
        print('all files in archive', mat.files)
        x, sf = mat['x'], 512
        n_elec, n_pts, n_trials = x.shape
        print ('--> compute power on : ', rep +'/'+ fi,'shape', x.shape, 'baseline:', baseline)

        # Compute and save power objects :
        powObj_x = power(sf, n_pts,**kwargs, method='hilbert')
        win_all, time = _manageWindow(x.shape[1], width=width, step=step)
        kwargs['fname'] = fname
        kwargs['labels'], kwargs['channels']= mat['Mai_RL'], mat['channels']
        kwargs['xyz'] = mat['xyz']
        xpow = powObj_x.get(x,n_jobs=-1)[0]
        time_down = resample(np.array(time),down=51)
        xpow_down = resample(xpow,down=51,axis=-2)
        kwargs['time'], kwargs['time_ds'] = np.array(time) / sf, np.array(time_down) / sf 
        kwargs['xpow'], kwargs['xpow_down'] = xpow, xpow_down
        save_x = path2save+ fi.replace('bipo_all_noWM_physFT.npz', 'bipo_all_'+code+'_6freqs_ds=6.npz')
        np.savez(save_x, **kwargs)
        del kwargs['xpow'],  kwargs['xpow_down'], kwargs['fname']
        del kwargs, x, sf, n_elec, n_trials

-> Olfacto loaded
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/CHAF_odor_1_bipo_all_noWM_physFT.npz shape (61, 3584, 1) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/CHAF_odor_2_bipo_all_noWM_physFT.npz shape (61, 3584, 3) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/LEFC_odor_14_bipo_all_noWM_physFT.npz shape (27, 3584, 6) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/LEFC_odor_15_bipo_all_noWM_physFT.npz shape (27, 3584, 11) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odo

all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/CHAF_odor_3_bipo_all_noWM_physFT.npz shape (61, 3584, 3) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/CHAF_odor_4_bipo_all_noWM_physFT.npz shape (61, 3584, 1) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/FERJ_odor_16_bipo_all_noWM_physFT.npz shape (32, 3584, 3) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/FERJ_odor_17_bipo_all_noWM_physFT.npz shape (32, 3584, 9) baseline: No baseline
all files in archive ['labels', 'x', 'aal', 'xyz', 'channels', 'sf', 'Mai', 'BA', 'Mai_RL']
--> compute power on :  Encoding_By_Odor/LEFC_odor_2_bipo_

#### Concatenate Power across all ODORS

In [74]:
import numpy as np
from brainpipe.system import study
from utils import su_list_od
from os.path import join, exists
from os import makedirs
from itertools import product

st = study('Olfacto')
PATH = join(st.path, 'feature/TPSim_power_data/')
PATH_OD = join(PATH, 'Power_all_elecs_E_R_by_odor_all_elecsFT/')
od_name = join(PATH_OD, '{}_odor_{}_bipo_all_E_6freqs_ds=6.npz')
PATH_COND = join(PATH, 'Power_all_elecs_E_R_all_odors_v=1_elecs=all/')
save_name = join(PATH_COND, '{}_all_odors_6freqs_hilbert=ok_ds=6.npz')

if not exists(PATH_COND):
    makedirs(PATH_COND)
    
for su in su_list_od:
    pow_data = np.array([])
    for od in su_list_od[su]:
        print('>>> processing:', su, od)
        mat = np.load(od_name.format(su,od),allow_pickle=True)
        pow_data = np.concatenate((pow_data, mat['xpow']),axis=-1) \
                                        if np.size(pow_data) else mat['xpow']
    dict_pow = {}
    for file in mat.files:
        dict_pow[file] = mat[file]
    dict_pow['xpow'] = pow_data
    np.savez(save_name.format(su),**dict_pow)
    print(dict_pow['xpow'].shape)

-> Olfacto loaded
>>> processing: CHAF 1
>>> processing: CHAF 2
>>> processing: CHAF 4
>>> processing: CHAF 5
>>> processing: CHAF 3
>>> processing: CHAF 8
>>> processing: CHAF 7
>>> processing: CHAF 9
(4, 61, 597, 21)
>>> processing: VACJ 11
>>> processing: VACJ 14
>>> processing: VACJ 12
>>> processing: VACJ 10
>>> processing: VACJ 15
>>> processing: VACJ 17
>>> processing: VACJ 16
>>> processing: VACJ 13
(4, 39, 597, 22)
>>> processing: SEMC 7
>>> processing: SEMC 10
>>> processing: SEMC 11
>>> processing: SEMC 12
>>> processing: SEMC 13
>>> processing: SEMC 5
>>> processing: SEMC 8
>>> processing: SEMC 9
(4, 53, 597, 40)
>>> processing: PIRJ 1
>>> processing: PIRJ 9
>>> processing: PIRJ 5
>>> processing: PIRJ 4
>>> processing: PIRJ 6
>>> processing: PIRJ 7
>>> processing: PIRJ 18
(4, 18, 597, 43)
>>> processing: LEFC 15
>>> processing: LEFC 2
>>> processing: LEFC 1
>>> processing: LEFC 16
>>> processing: LEFC 14
>>> processing: LEFC 3
>>> processing: LEFC 4
>>> processing: LEFC 17


In [79]:
from scipy.stats import wilcoxon
from mne.stats import bonferroni_correction, fdr_correction
import pandas as pd

st = study('Olfacto')
PATH_FEAT = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_all_odors_v=1_elecs=all//')
PATH_SAVE = join(PATH_FEAT, 'Wilcox_pre_post_avg/')
if not exists(PATH_SAVE):
    makedirs(PATH_SAVE)
feature, corr, thr = 'xpow', 'fdr', 0.05
fnames = ['low_theta', 'high_theta', 'beta', 'BBG']

"""
This script compare pre and post average power using Wilcoxon non-parametric Ttest
Results are computed by freq band and corrected across all electrodes (all subj together)
"""

def create_stats_df(T,p,corr,thr,chans,labs,subjs,freq,x,y,z,all_pre, all_post):

    #create df ALL elecs
    chans_sig, su_sig, lab_sig = np.array(chans), np.array(subjs), np.array(labs)
    Tvals, pvals, x, y, z = np.array(T), np.array(p), np.array(x), np.array(y), np.array(z)
    all_pre, all_post = np.array(all_pre), np.array(all_post)
    df = pd.DataFrame({'subjects':su_sig, 'channels':chans_sig, 'x':x, 'y':y,'z':z,
                'labels':lab_sig,'Tvals':Tvals, 'pvals':pvals, 'pre':all_pre, 'post':all_post})
    _, pvals_corr = bonferroni_correction(df['pvals'].values,thr) if corr == 'bonf' \
                                                    else fdr_correction(df['pvals'].values,thr)
    df['p_corr'] = pvals_corr
    df_not_corr = df.loc[df['pvals']<0.05]
    print('total nb of sig elec', freq, df_not_corr.shape[0], 'on ', df.shape[0],'in total')
    print(df_not_corr.groupby(['labels','subjects']).count()['Tvals'])
    #print(np.unique(df_not_corr['labels']))
    #df_not_corr.to_csv(PATH_SAVE+'Wilcox_odor_responsive_ALL_{}_corr={}_thr={}.csv'.format(freq,
    #                    'unc_p',str(thr)),index=False)

    #create signif df
    df_pos = df.loc[(df['pvals']<thr)&(df['Tvals']>0)]
    df_neg = df.loc[(df['pvals']<thr)&(df['Tvals']<0)]
    print('total nb of sig elec', freq, 'pos',df_pos.shape[0], 'neg', df_neg.shape[0])
    #print(df_pos.groupby(['labels','subjects']).count()['Tvals'])
    print('')
    if df_pos.shape[0] > 0:
        df_pos.to_csv(PATH_SAVE+'/Wilcox_odor_responsive_{}_corr={}_thr={}.csv'.format(freq,
                            corr,str(thr)),index=False)

if __name__ == "__main__":

    for freq in fnames:
        all_Tvals, all_pvals = [], []
        all_chans, all_labels, subj_list = [], [], []
        x, y, z, all_pre, all_post = [], [], [], [], []
        for su in su_list_od:
            mat = np.load(PATH_FEAT+su+'_all_odors_6freqs_hilbert=ok_ds=6.npz', allow_pickle=True)
            id_f = [i for i,f in enumerate(mat['fname']) if f==freq][0]
            data = mat[feature][id_f,...]
            nchans, npts, ntrials = data.shape
            su_list = [su]*nchans
            time = mat['time'] - 3
            bsl_id = [i for i,t in enumerate(time) if -0.5<=t<=-0.1]
            targ_id = [i for i,t in enumerate(time) if 0<=t<=2]
            channels, labels, xyz = mat['channels'], mat['labels'], mat['xyz']
            # baseline -1000 to -300 ms // odor 0 to +2000ms (check mat['time'])
            pre, post = np.mean(data[:,bsl_id,:], axis=1), np.mean(data[:,targ_id,:],axis=1)
            all_pre.extend(np.mean(pre,axis=1)), all_post.extend(np.mean(post,axis=1))

            T_su, p_su = [], []
            for n in range(nchans):
                T, p = wilcoxon(pre[n],post[n])
                T_su.append(T), p_su.append(p)
            all_Tvals.extend(T_su), all_pvals.extend(p_su)
            all_chans.extend(channels), all_labels.extend(labels), subj_list.extend(su_list)
            x.extend(xyz[:,0]), y.extend(xyz[:,1]), z.extend(xyz[:,2])

        print('processing {} using {} correction'.format(freq, corr))
        create_stats_df(all_Tvals, all_pvals, corr,thr,all_chans,
                        all_labels,subj_list,freq,x,y,z, all_pre, all_post)


-> Olfacto loaded
processing low_theta using fdr correction
total nb of sig elec low_theta 27 on  230 in total
labels   subjects
IFG      SEMC        4
         VACJ        1
MFG      CHAF        1
         LEFC        3
         SEMC        3
         VACJ        2
OFC_olf  LEFC        1
         SEMC        1
         VACJ        1
SFG      CHAF        1
         SEMC        2
aHC      FERJ        1
         SEMC        1
         VACJ        2
pPirT    FERJ        2
         VACJ        1
Name: Tvals, dtype: int64
total nb of sig elec low_theta pos 27 neg 0

processing high_theta using fdr correction
total nb of sig elec high_theta 32 on  230 in total
labels   subjects
ACC      CHAF        1
         SEMC        1
IFG      SEMC        1
         VACJ        2
Ins_olf  FERJ        5
MFG      CHAF        1
         SEMC        3
OFC_olf  LEFC        2
         PIRJ        1
         SEMC        3
PHG      PIRJ        1
SFG      CHAF        2
         SEMC        2
aHC      FERJ       